In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
# Set random seeds for reproducibility
from transformers import set_seed
SEED = 355
torch.manual_seed(SEED)
set_seed(SEED)

model = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model, device_map="auto")

input_text = "1. If I have 23 apples and I give 7 to my friend and sell the rest for $1.33 each, how much money do I have?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_new_tokens=200, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id, do_sample=True)

for i, sample_output in enumerate(output):
    print(f"########\nOUTPUT {i}:\n {tokenizer.decode(sample_output[len(input_ids[0]):], skip_special_tokens=True)}\n")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


########
OUTPUT 0:
  Let's think through this step-by-step:

1. You start with 23 apples.
2. You give away 7 apples: 23 - 7 = 16 apples left.
3. You then sell these remaining apples for $1.33 each: 16 * $1.33 = $21.84
4. So you end up with $21.84.

Therefore, after giving 7 apples to your friend and selling the rest at $1.33 each, you would have $21.84 in total. 

The answer is $21.84. (rounded to two decimal places)

########
OUTPUT 1:
  

To determine how much money you would have after selling the remaining apples, we need to follow these steps:

1. Calculate the number of apples left after giving some to your friend.
2. Determine the total revenue from selling the remaining apples at $1.33 each.

**Step 1: Calculate the number of apples left after giving 7 to your friend.**

You start with 23 apples and give away 7:
\[ 23 - 7 = 16 \]
So, you have 16 apples left.

**Step 2: Determine the total revenue from selling the remaining apples.**

Each apple is sold for $1.33, so the total r

First, let's make sure we're using the appropriate chat template for this model.

In [3]:
torch.manual_seed(SEED)
set_seed(SEED)

input_messages = [{"role":"system", "content":"Answer the user's question."},
                  {"role":"user", "content":input_text}]
input_ids = tokenizer.apply_chat_template(input_messages, add_generation_prompt=True,return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_new_tokens=200, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id, do_sample=True)

for i, sample_output in enumerate(output):
    print(f"########\nOUTPUT {i}:\n {tokenizer.decode(sample_output[len(input_ids[0]):], skip_special_tokens=True)}\n")


########
OUTPUT 0:
 If you start with 23 apples and give away 7, you would have:

23 - 7 = 16 apples left

Then you sell these 16 apples at $1.33 each:

16 * $1.33 = $21.84

So after selling the apples, you would have $21.84.

########
OUTPUT 1:
 You start with 23 apples and give away 7 to your friend. So you have:

23 - 7 = 16 apples left.

Then you sell these 16 apples at $1.33 each:

16 * $1.33 = $21.84

So after selling the apples, you would have $21.84 in total.

########
OUTPUT 2:
 You start with 23 apples. After giving away 7 to your friend, you have 23 - 7 = 16 apples left. If you sell these 16 apples at $1.33 each, you would earn 16 * $1.33 = $21.88. So, you would have a total of $21.88.

########
OUTPUT 3:
 You would have 23 - 7 = 16 apples left after giving some to your friend. You would then have 16 * $1.33 = $21.48 remaining. So you would have a total of $21.48 + $1.33 = $22.81.

########
OUTPUT 4:
 You gave your friend 7 apples, so you had 23 - 7 = 16 apples left. You sol

In [4]:
torch.manual_seed(SEED)
set_seed(SEED)

input_messages = [{"role":"system", "content":"Answer the user's question. Think step-by-step and double-check your arithmetic. Enclose your final answer in delimiters like `<answer>{your answer here}</answer>`."},
                  {"role":"user", "content":input_text}]
input_ids = tokenizer.apply_chat_template(input_messages, add_generation_prompt=True,return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_new_tokens=300, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id, do_sample=True)

for i, sample_output in enumerate(output):
    print(f"########\nOUTPUT {i}:\n {tokenizer.decode(sample_output[len(input_ids[0]):], skip_special_tokens=True)}\n")

########
OUTPUT 0:
 To solve this problem, let's break it down into steps:

1. First, we need to determine how many apples are left after giving 7 to a friend:
   - Initial amount of apples: 23
   - Apples given to friend: 7
   - Remaining apples: 23 - 7 = 16

2. Now, we calculate the total revenue from selling these remaining apples at $1.33 each:
   - Price per apple: $1.33
   - Number of remaining apples: 16
   - Total revenue: 16 * $1.33 = $21.88

Therefore, after selling the apples, you would make $21.88.

The final answer is: <answer>$21.88</answer>.

########
OUTPUT 1:
 Let's break this down step by step:

1. **Initial amount**: You start with 23 apples.
   
2. **Apples given away**: You give 7 apples to your friend.

   Calculation: \( 23 - 7 = 16 \) apples left.

3. **Sale of remaining apples**: You decide to sell these 16 apples at a price of $1.33 each.

   Calculation: \( 16 \times 1.33 = 21.12 \)

4. **Total earnings from selling**: You earn $21.12 from selling the apples.

In [5]:
# Examples for FSL
example_1_question = "If you have 41 pizzas and each pizza has 8 slices, and you sell the slices for 2.15 each, how much money do you make?"
example_1_answer = """Let's solve this step by step, breaking down arithmetic steps to make them easier:

Calculate the total number of slices: 41 * 8 = (40 * 8) + (1 * 8) = 320 + 8 = 328 slices
Calculate the total money made: 328 * 2.15 = (300 * 2.15) + (20 * 2.15) + (8 * 2.15) = ((300 * 2) + (300 * 0.15)) + ((20 * 2) + (20 * 0.15)) + ((8 * 2) + (8 * 0.15)) = (600 + 45) + (40 + 3) + (16 + 1.2) = 645 + 43 + 17.2 = 705.2
<answer>705.2</answer>"""
example_2_question = "If Sandy has 61 acorns, gives 7 to Bob, and then sells the remaining acorns for $1.20 per acorn, how much money does she make?"
example_2_answer = """Let's solve this step by step, breaking down arithmetic steps to make them easier:

Calculate the acorns left after giving some to Bob: 61 - 7 = 54 acorns
Calculate the total money made: 54 * 1.20 = (50 * 1.20) + (4 * 1.20) = ((50 + 1) + (50 * 0.20)) + ((4 * 1) + (4 * 0.20)) = (60 + 10) + (4 + 0.8) = 70 + 4.8 = 74.8 
<answer>74.8</answer>"""

In [6]:
torch.manual_seed(SEED)
set_seed(SEED)

input_messages = [{"role":"system", "content":"Answer the user's question. Think step-by-step and double-check your arithmetic. Enclose your final answer in delimiters like `<answer>{your answer here}</answer>`."},
                    {"role":"user", "content":example_1_question},
                    {"role":"assistant", "content":example_1_answer},
                    {"role":"user", "content":example_2_question},
                    {"role":"assistant", "content":example_2_answer},
                    {"role":"user", "content":input_text}]
input_ids = tokenizer.apply_chat_template(input_messages, add_generation_prompt=True,return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_new_tokens=200, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id, do_sample=True)

for i, sample_output in enumerate(output):
    print(f"########\nOUTPUT {i}:\n {tokenizer.decode(sample_output[len(input_ids[0]):], skip_special_tokens=True)}\n")

########
OUTPUT 0:
 To calculate the total money you would earn from selling the apples, we first need to determine how many apples you sold.

You started with 23 apples and gave away 7, so you sold:
23 - 7 = 16 apples

Now that you've sold 16 apples, we can calculate the total amount of money you would earn by multiplying the number of apples sold by the price per apple:

16 apples * $1.33/apple = $21.28

Therefore, if you sell all 16 apples at $1.33 each, you would have earned $21.28.

########
OUTPUT 1:
 Let's break it down step-by-step:

1. **Initial amount:** You start with 23 apples.
   \[
   \text{Initial amount} = 23
   \]

2. **Apples given away:** You give 7 apples to your friend.
   \[
   \text{Apples given away} = 7
   \]

3. **Remaining apples:** Subtract the given apples from the initial amount.
   \[
   \text{Remaining apples} = 23 - 7 = 16
   \]

4. **Price per apple:** The price is $1.33.

5. **Total money made:** Multiply the remaining apples by the price per apple.
 

## RAG Example  

In [7]:
from langchain_community.document_loaders import UnstructuredXMLLoader

loader = UnstructuredXMLLoader(
    "./can_tax.xml",
)

docs = loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=8192, chunk_overlap=1024)

chunked_docs = splitter.split_documents(docs)

In [9]:
print(F"NUMBER OF CHUNKS: {len(chunked_docs)}\n\n")
print(chunked_docs[1].page_content)

NUMBER OF CHUNKS: 933


A taxpayer’s loss for a taxation year from an office or employment is the amount of the taxpayer’s loss, if any, for the taxation year from that source computed by applying, with such modifications as the circumstances require, the provisions of this Act respecting the computation of income from that source.

[NOTE: Application provisions are not included in the consolidated text; see relevant amending Acts and regulations.] 

1970-71-72, c. 63, s. 1“5”

Inclusions

Amounts to be included as income from office or employment

6

(1)

There shall be included in computing the income of a taxpayer for a taxation year as income from an office or employment such of the following amounts as are applicable

Value of benefits

(a)

the value of board, lodging and other benefits of any kind whatever received or enjoyed by the taxpayer, or by a person who does not deal at arm’s length with the taxpayer, in the year in respect of, in the course of, or by virtue of the taxpa

In [10]:
from sentence_transformers import SentenceTransformer

st = SentenceTransformer("nomic-ai/modernbert-embed-base")
st.to("cuda")
chunked_docs_embeddings = st.encode([chunked_doc.page_content for chunked_doc in chunked_docs], convert_to_tensor=True)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
/home/cehrett/.conda/envs/LLMsFT/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:167: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

hfe = HuggingFaceEmbeddings(model_name="nomic-ai/modernbert-embed-base")
hfe

/local_scratch/slurm.1895134/ipykernel_2700173/3766319925.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hfe = HuggingFaceEmbeddings(model_name="nomic-ai/modernbert-embed-base")


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: ModernBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='nomic-ai/modernbert-embed-base', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from sentence_transformers import SentenceTransformer

# Load a Sentence Transformer model 
model_name = "nomic-ai/modernbert-embed-base"
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={'device': 'cuda'})  

# Create a FAISS vectorstore using the Sentence Transformer embeddings
vectorstore = FAISS.from_documents(chunked_docs, embeddings) 

/local_scratch/slurm.2204086/ipykernel_3658335/3255990169.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={'device': 'cuda'})


In [14]:
# Perform a similarity search on a query 
query = "I'm a schoolteacher, can I deduct expenses for my classroom?"
results = vectorstore.similarity_search(query, k=2)  # Get top 2 similar documents

# Print the results
for i,result in enumerate(results):
    print(f"RESULT {i}:\n{result.page_content}\n\n")

RESULT 0:
(ii)

directly consumed or used in the performance of the duties of the eligible educator’s employment; and

(b)

dépense admissible

the eligible educator is not entitled to receive a reimbursement, allowance or any other form of assistance (other than an amount that is included in computing the income for any taxation year of the eligible educator and that is not deductible in computing the taxable income of the eligible educator) in respect of the amount paid. (

return of income

déclaration de revenu

teaching supplies

(a)

consumable supplies; and

(b)

fournitures scolaires

prescribed durable goods. (

Deemed overpayment

(2)

An eligible educator who files a return of income for a taxation year and who makes a claim under this subsection is deemed to have paid, at the end of the year, on account of tax payable under this Part for the year, an amount equal to 25% of the least of

(a)

$1,000;

(b)

the total of all amounts each of which is an eligible supplies expens

In [15]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 2}
)

In [16]:
system_prompt = "Give the below context from the Canadian tax code, answer the user's question. Make sure that your answer is based solely in the context provided below.\n\n## Context:\n{context}"
user_prompt = "I'm a schoolteacher, can I deduct expenses for my classroom?"
context_docs = '/n'.join([doc.page_content for doc in retriever.get_relevant_documents(query)])

prompt_template = [{"role":"system", "content":system_prompt.format(context=context_docs)}, {"role":"user", "content":user_prompt}]

input_ids = tokenizer.apply_chat_template(prompt_template, add_generation_prompt=True,return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_new_tokens=200, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id, do_sample=True)

for i, sample_output in enumerate(output):
    print(f"########\nOUTPUT {i}:\n {tokenizer.decode(sample_output[len(input_ids[0]):], skip_special_tokens=True)}\n")



/local_scratch/slurm.2204086/ipykernel_3658335/959300657.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = '/n'.join([doc.page_content for doc in retriever.get_relevant_documents(query)])


########
OUTPUT 0:
 Yes, you can deduct expenses for your classroom as long as they fall within the defined categories and are directly consumed or used in the performance of the duties of the eligible educator's employment. The specific requirements will depend on the details of the claim, but generally, expenses must be directly associated with teaching activities and meet certain criteria.

########
OUTPUT 1:
 Yes, you can deduct expenses for your classroom if they fall under the "dépenses admissibles" category specified in the given context. The context states that "dépenses admissibles" refer to "dépenses admissible", which means "admissible expenses". This allows for deductions that are directly consumed or used in the performance of the duties of the eligible educator's employment. If you meet these criteria, you can claim deductions for expenses related to your classroom activities.

########
OUTPUT 2:
 Yes, you can deduct expenses for your classroom. According to the provided 